In [1]:
import datasets
from transformers import (
    HfArgumentParser,
    TrainingArguments,
)

# local
from examples.lgvit_utils import (
    ModelArguments,
    DataTrainingArguments,
)

from models.deit_highway import DeiTImageProcessor, DeiTConfig, DeiTHighwayForImageClassification


/home/iony/miniconda3/envs/lgvit/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
args = [
"--run_name" ,"DeiT_confidence_LGViT__LGViT",
"--image_processor_name" ,"facebook/deit-base-distilled-patch16-224",
"--config_name" ,"facebook/deit-base-distilled-patch16-224",
"--model_name_or_path" ,"/home/iony/DTU/f24/thesis/code/lgvit/LGViT-ViT-Cifar100",
"--dataset_name" ,"uoft-cs/cifar100",
"--output_dir" ,"../outputs/DeiT-base/uoft-cs/cifar100/LGViT/confidence/",
"--remove_unused_columns" ,"False",
"--backbone" ,"DeiT",
"--exit_strategy" ,"confidence",
"--do_train" ,"False",
"--do_eval", "True",
"--per_device_eval_batch_size" ,"1",
"--seed" ,"777",
"--report_to" ,"wandb",
"--use_auth_token" ,"False",
"--ignore_mismatched_sizes" ,"True",
]
model_args, data_args, training_args = parser.parse_args_into_dataclasses(args)

task_arg = datasets.ImageClassification(image_column='img', label_column='fine_label')

dataset = datasets.load_dataset(
    path=data_args.dataset_name,
    name=data_args.dataset_config_name,
    cache_dir=model_args.cache_dir,
    task=task_arg,
    use_auth_token=True if model_args.use_auth_token else None,
    # ignore_verifications=True,
)

# Prepare label mappings.
# We'll include these in the model's config to get human readable labels in the Inference API.
labels = dataset["train"].features["labels"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

# only eval case
config = DeiTConfig.from_pretrained(
    model_args.model_name_or_path,
    num_labels=len(labels),
    label2id=label2id,
    id2label=id2label,
    finetuning_task="image-classification",
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    backbone=model_args.backbone,
    threshold=model_args.threshold,
    exit_strategy=model_args.exit_strategy,
    # train_strategy=model_args.train_strategy,
    # num_early_exits=model_args.num_early_exits,
    # position_exits=model_args.position_exits,
    # highway_type=model_args.highway_type,
    # loss_coefficient=model_args.loss_coefficient,
    # homo_loss_coefficient=model_args.homo_loss_coefficient,
    # hete_loss_coefficient=model_args.hete_loss_coefficient,
    # feature_loss_coefficient=model_args.feature_loss_coefficient,
    # output_hidden_states=model_args.output_hidden_states,
    # use_auth_token=True if model_args.use_auth_token else None,
)

total_optimization_steps = int(len(dataset['train']) // training_args.per_device_train_batch_size * training_args.num_train_epochs)
config.total_optimization_steps = total_optimization_steps

model = DeiTHighwayForImageClassification.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    train_highway=True,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    # use_auth_token=True if model_args.use_auth_token else None,
    ignore_mismatched_sizes=model_args.ignore_mismatched_sizes,
)


/home/iony/miniconda3/envs/lgvit/lib/python3.9/site-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/home/iony/miniconda3/envs/lgvit/lib/python3.9/site-packages/datasets/load.py:2102: FutureWarning: 'task' was deprecated in version 2.13.0 and will be removed in 3.0.0.

  warnings.warn(
/home/iony/miniconda3/envs/lgvit/lib/python3.9/site-packages/transformers/modeling_utils.py:415: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `Tru

backbone:DeiT
exit_type:LGViT
use_lte:False
The exits are in position: [4, 5, 6, 7, 8, 9, 10, 11]


Some weights of the model checkpoint at /home/iony/DTU/f24/thesis/code/lgvit/LGViT-ViT-Cifar100 were not used when initializing DeiTHighwayForImageClassification: ['classifier.weight', 'deit.encoder.highway.3.classifier.weight', 'deit.encoder.highway.7.classifier.bias', 'deit.encoder.highway.5.classifier.bias', 'deit.encoder.highway.4.classifier.bias', 'deit.encoder.highway.1.classifier.bias', 'deit.encoder.highway.0.classifier.bias', 'deit.encoder.highway.5.classifier.weight', 'deit.encoder.highway.3.classifier.bias', 'deit.encoder.highway.0.classifier.weight', 'deit.encoder.highway.2.classifier.weight', 'deit.encoder.highway.7.classifier.weight', 'deit.encoder.highway.2.classifier.bias', 'deit.encoder.highway.1.classifier.weight', 'deit.encoder.highway.6.classifier.weight', 'deit.encoder.highway.6.classifier.bias', 'deit.encoder.highway.4.classifier.weight', 'classifier.bias']
- This IS expected if you are initializing DeiTHighwayForImageClassification from the checkpoint of a model 

In [10]:
X = dataset['train'][0]
y = model(X["image"])

print(X["image"].shape)
print(y.shape)

AttributeError: 'PngImageFile' object has no attribute 'dtype'

In [ ]:
# Passing single batch to model
X = next(iter(train_dataloader))
y = model(X["pixel_values"])

print(X["pixel_values"].shape)
print(y.shape)